In [ ]:
cd ..

In [ ]:
import echofilter.raw.loader

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
ROOT_DATA_DIR = "/media/scott/scratch/Datasets/dsforce/surveyExports"
# example with 1 passive period, 1 turbulence cut out
sample = "mobile/Survey16/Survey16_GR3_N1W_E"
# example with lots of short passive periods, 1 proper passive period, 1 turbulence cut out
# sample = 'mobile/Survey17/Survey17_GR1_N0W_E'

fname_raw = os.path.join(ROOT_DATA_DIR, sample + "_Sv_raw.csv")
fname_masked = os.path.join(ROOT_DATA_DIR, sample + "_Sv.csv")

ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(fname_raw)
ts_mskd, depths_mskd, signals_mskd = echofilter.raw.loader.transect_loader(fname_masked)

t_bot, d_bot = echofilter.raw.loader.evl_loader(
    os.path.join(ROOT_DATA_DIR, sample + "_bottom.evl")
)
t_top, d_top = echofilter.raw.loader.evl_loader(
    os.path.join(ROOT_DATA_DIR, sample + "_turbulence.evl")
)

In [ ]:
ts_new, d_top_new, d_bot_new = echofilter.raw.manipulate.make_lines_from_masked_csv(
    fname_masked
)

In [ ]:
np.min(signals_raw), np.max(signals_raw)

In [ ]:
ts_raw.shape, depths_raw.shape, signals_raw.shape

In [ ]:
turbulence_color = "c"
bottom_color = "#00ee00"

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, signals_raw.T)
plt.plot(t_top, d_top, "k")
plt.plot(t_bot, d_bot, "w")
plt.plot(ts_new, d_top_new, turbulence_color)
plt.plot(ts_new, d_bot_new, bottom_color)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, signals_mskd.T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.show()

## Finding rows which are fully removed from masked output

In [ ]:
np.all(np.isnan(signals_mskd), axis=1)

In [ ]:
allnan = np.all(np.isnan(signals_mskd), axis=1)
removed_starts = np.nonzero(np.diff(allnan.astype(np.float)) > 0)[0] + 1
removed_ends = np.nonzero(np.diff(allnan.astype(np.float)) < 0)[0] + 1

if removed_ends[0] < removed_starts[0]:
    removed_starts = np.concatenate(([0], removed_starts))

if removed_starts[-1] > removed_ends[-1]:
    removed_ends = np.concatenate((removed_ends, [len(allnan)]))

In [ ]:
print(removed_starts)
print(removed_ends)

In [ ]:
for index_start, index_end in zip(removed_starts, removed_ends):
    for ts, depths, signals in (
        (ts_mskd, depths_mskd, signals_mskd),
        (ts_raw, depths_raw, signals_raw),
    ):
        an = np.all(np.isnan(signals[index_start:index_end, :]), axis=1)
        print(np.sum(an), np.sum(~an))
        plt.figure(figsize=(12, 12))
        plt.pcolormesh(
            ts[index_start:index_end], depths, signals[index_start:index_end, :].T
        )
        li = np.all([t_top >= ts[index_start], t_top <= ts[index_end - 1]], axis=0)
        plt.plot(t_top[li], d_top[li], "k")
        li = np.all([t_bot >= ts[index_start], t_bot <= ts[index_end - 1]], axis=0)
        plt.plot(t_bot[li], d_bot[li], "w")
        plt.plot(
            ts_new[index_start:index_end],
            d_top_new[index_start:index_end],
            turbulence_color,
        )
        plt.plot(
            ts_new[index_start:index_end],
            d_bot_new[index_start:index_end],
            bottom_color,
        )
        plt.gca().invert_yaxis()
        plt.show()

In [ ]:
for index_start, index_end in zip(
    np.concatenate(([0], removed_ends)),
    np.concatenate((removed_starts, [signals_raw.shape[0]])),
):
    if index_start == index_end:
        continue
    for ts, depths, signals in (
        (ts_mskd, depths_mskd, signals_mskd),
        (ts_raw, depths_raw, signals_raw),
    ):
        an = np.all(np.isnan(signals[index_start:index_end, :]), axis=1)
        print(np.sum(an), np.sum(~an))
        plt.figure(figsize=(12, 12))
        plt.pcolormesh(
            ts[index_start:index_end], depths, signals[index_start:index_end, :].T
        )
        li = np.all([t_top >= ts[index_start], t_top <= ts[index_end - 1]], axis=0)
        plt.plot(t_top[li], d_top[li], "k")
        li = np.all([t_bot >= ts[index_start], t_bot <= ts[index_end - 1]], axis=0)
        plt.plot(t_bot[li], d_bot[li], "w")
        plt.plot(
            ts_new[index_start:index_end],
            d_top_new[index_start:index_end],
            turbulence_color,
        )
        plt.plot(
            ts_new[index_start:index_end],
            d_bot_new[index_start:index_end],
            bottom_color,
        )
        plt.gca().invert_yaxis()
        plt.show()

In [ ]:
print(signals_raw.shape)
print(signals_mskd.shape)

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw[234:257], depths_raw[:34], signals_raw[234:257, :34].T)
plt.gca().invert_yaxis()
plt.show()

plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw[235:256], depths_raw[:34], signals_raw[235:256, :34].T)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd[234:257], depths_mskd[:34], signals_mskd[234:257, :34].T)
plt.gca().invert_yaxis()
plt.show()

plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd[235:256], depths_mskd[:34], signals_mskd[235:256, :34].T)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd[234:257], depths_mskd, signals_mskd[234:257, :].T)
plt.gca().invert_yaxis()
plt.show()

plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd[235:256], depths_mskd, signals_mskd[235:256, :].T)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
np.nonzero(allnan)